In [14]:
pip install -U spacy


In [15]:
!python -m spacy download en_core_web_md


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 53.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [16]:
import spacy
from spacy import displacy

# Tải mô hình tiếng Anh đã cài đặt
# Sử dụng en_core_web_md vì nó chứa các vector từ và cây cú pháp đầy đủ
nlp = spacy.load("en_core_web_md")

# Câu ví dụ
text = "The quick brown fox jumps over the lazy dog."

# Phân tích câu
doc = nlp(text)


# Tùy chọn để hiển thị trong trình duyệt
# options = {"compact": True, "color": "blue", "font": "Source Sans Pro"}
# Khởi chạy server tại http://127.0.0.1:5000
# Bạn có thể truy cập địa chỉ này trên trình duyệt để xem cây phụ thuộc
# Nhấn Ctrl+C trong terminal để dừng server
displacy.serve(doc, style="dep")





Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


- Từ “jumps” là ROOT.
- Các dependent của “jumps”:fox (nsubj); over (prep) 
- fox là head của các từ:The (det — mạo từ) ;quick (amod — tính từ bổ nghĩa); brown (amod — tính từ bổ nghĩa)

### Phần 3

In [25]:
# Lấy một câu khác để phân tích
text = "Apple is looking at buying U.K. startup for $1 billion"
doc = nlp(text)
# In ra thông tin của từng token
print(f"{'TEXT':<12} | {'DEP':<10} | {'HEAD TEXT':<12} | {'HEAD POS':<8} | {'CHILDREN'}")
print("-" * 70)

for token in doc:
  # Trích xuất các thuộc tính
  children = [child.text for child in token.children]
  print(f"{token.text:<12} | {token.dep_:<10} | {token.head.text:<12} | {token.head.pos_:<8} | {children}")


TEXT         | DEP        | HEAD TEXT    | HEAD POS | CHILDREN
----------------------------------------------------------------------
Apple        | nsubj      | looking      | VERB     | []
is           | aux        | looking      | VERB     | []
looking      | ROOT       | looking      | VERB     | ['Apple', 'is', 'at']
at           | prep       | looking      | VERB     | ['buying']
buying       | pcomp      | at           | ADP      | ['startup']
U.K.         | compound   | startup      | NOUN     | []
startup      | dobj       | buying       | VERB     | ['U.K.', 'for']
for          | prep       | startup      | NOUN     | ['billion']
$            | quantmod   | billion      | NUM      | []
1            | compound   | billion      | NUM      | []
billion      | pobj       | for          | ADP      | ['$', '1']


### Phần 4

In [26]:
text = "The cat chased the mouse and the dog watched them."
doc = nlp(text)

for token in doc:
    # Chỉ tìm các động từ
    if token.pos_ == "VERB":
        verb = token.text
        subject = ""
        obj = ""
        # Tìm chủ ngữ (nsubj) và tân ngữ (dobj) trong các con của động từ
        for child in token.children:
            if child.dep_ == "nsubj":
                subject = child.text
            if child.dep_ == "dobj":
                obj = child.text

        if subject and obj:
            print(f"Found Triplet: ({subject}, {verb}, {obj})")


Found Triplet: (cat, chased, mouse)
Found Triplet: (dog, watched, them)


In [27]:
text = "The big, fluffy white cat is sleeping on the warm mat."
doc = nlp(text)

for token in doc:
    # Chỉ tìm các danh từ
    if token.pos_ == "NOUN":
        # Tìm các tính từ bổ nghĩa (amod) trong các con của danh từ
        adjectives = [child.text for child in token.children if child.dep_ == "amod"]

        if adjectives:
            print(f"Danh từ '{token.text}' được bổ nghĩa bởi: {adjectives}")


Danh từ 'cat' được bổ nghĩa bởi: ['big', 'fluffy', 'white']
Danh từ 'mat' được bổ nghĩa bởi: ['warm']


### Phần 5: Bài tập tự luyện

- Bài 1: Tìm động từ chính của câu

In [28]:
def find_main_verb(doc):
    for token in doc:
        if token.dep_ == "ROOT":
            return token
    return None

doc = nlp("The quick brown fox jumps over the lazy dog.")
print(find_main_verb(doc))


jumps


- Bài 2: Trích xuất các cụm danh từ (Noun Chunks)

In [29]:
def extract_noun_chunk(token):
    words = []

    # children bổ nghĩa (det, amod, compound)
    for child in token.children:
        if child.dep_ in ("det", "amod", "compound"):
            words.append(child.text)
    words.append(token.text)
    return " ".join(words)

def find_noun_chunks(doc):
    chunks = []
    for token in doc:
        if token.pos_ == "NOUN":
            chunks.append(extract_noun_chunk(token))
    return chunks

doc = nlp("The big white cat caught a small mouse.")
find_noun_chunks(doc)


['The big white cat', 'a small mouse']

- Bài 3: Tìm đường đi ngắn nhất trong cây

In [30]:
def get_path_to_root(token):
    path = [token]
    while token.head != token:
        token = token.head
        path.append(token)
    return path

doc = nlp("The big cat is chasing a mouse.")
token = doc[3]  # cat
[get.text for get in get_path_to_root(token)]


['is', 'chasing']